<a href="https://colab.research.google.com/github/El-Wally/Movie-Recommender/blob/main/MoviesRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing packages**

In [34]:
import pandas as pd


**Importing the dataset**

In [35]:
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 

--2022-07-10 15:59:17--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160301210 (153M) [application/zip]
Saving to: ‘moviedataset.zip’

moviedataset.zip    100%[===================>] 152.88M  34.8MB/s    in 5.1s    

2022-07-10 15:59:23 (30.2 MB/s) - ‘moviedataset.zip’ saved [160301210/160301210]

unziping ...
Archive:  moviedataset.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: tags.csv                


In [36]:
df = pd.read_csv('movies.csv')
df.head(50)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [37]:
df['title'][7001]

'Accident (1967)'

checking for null values

In [38]:
df.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [39]:
df1 = pd.read_csv('tags.csv')
df1.head(5)

,userId,movieId,tag,timestamp
0,19,2324,bittersweet,1428651158
1,19,2324,holocaust,1428651112
2,19,2324,World War II,1428651118
3,23,7075,hilarious,1378675786
4,23,7075,Underrated,1378675786


In [40]:
df1.describe

<bound method NDFrame.describe of         userId  movieId                            tag   timestamp
0           19     2324                    bittersweet  1428651158
1           19     2324                      holocaust  1428651112
2           19     2324                   World War II  1428651118
3           23     7075                      hilarious  1378675786
4           23     7075                     Underrated  1378675786
...        ...      ...                            ...         ...
586989  247751    96655                         future  1366041126
586990  247751    96655                         robots  1366041128
586991  247753      356                        classic  1430438776
586992  247753      356  oscar (best supporting actor)  1430438776
586993  247753      356                   social drama  1430438776

[586994 rows x 4 columns]>

In [41]:
df1['tag'].value_counts()

sci-fi                 6194
action                 4059
atmospheric            3839
based on a book        3799
comedy                 3745
                       ... 
Manny Coto                1
Darin Scott               1
Alain Berliner            1
Richard Friedenberg       1
great screenplays         1
Name: tag, Length: 45084, dtype: int64

In [42]:
df1[df1.isnull().any(axis=1)]

,userId,movieId,tag,timestamp
119651,44258,33826,NaN,1221450908
208021,91097,123,NaN,1199450867
208022,91097,346,NaN,1199451946
208026,91097,1184,NaN,1199452261
208033,91097,1785,NaN,1199452006
208034,91097,2194,NaN,1199450677
208036,91097,2691,NaN,1199451002
208044,91097,4103,NaN,1199451920
208046,91097,4473,NaN,1199451040
208048,91097,4616,NaN,1199452441


In [43]:
(df1['movieId']==1).sum()

548

In [44]:
  dff1 = df1[df1['movieId'].values  == 113]['tag']
  dff1=dff1.tolist()
  dff1

[]

In [45]:
df1.sort_values(by=['movieId'], inplace=True)
print(df1)
print(df1['tag'].iloc[0])

        userId  movieId          tag   timestamp
69201    29321        1       Disney  1273665156
585030  246887        1        pixar  1443906050
42826    15036        1        Pixar  1261495514
413751  173452        1        pixar  1445458804
413752  173452        1    Tim Allen  1445458849
...        ...      ...          ...         ...
308824  132513   151677       trippy  1453951839
308825  132513   151677         Yogi  1453951834
308819  132513   151677        drugs  1453951843
308818  132513   151677        dance  1453951844
411565  172056   151701  documentary  1454031135

[586994 rows x 4 columns]
Disney


In [46]:
#df1['movieId'].tolist()
x=list(set(df1['movieId']))

In [47]:
x.sort()
x

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 52,
 53,
 54,
 55,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 85,
 86,
 87,
 88,
 89,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 100,
 101,
 102,
 103,
 104,
 105,
 107,
 110,
 111,
 112,
 114,
 115,
 116,
 117,
 118,
 121,
 122,
 123,
 124,
 125,
 126,
 128,
 129,
 132,
 133,
 134,
 135,
 136,
 138,
 140,
 141,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 193,
 194,
 196,
 197,
 198,
 199,
 201,
 202,
 203,
 204,
 205,
 206

In [48]:
temp = 0
for i in x:
  dff1 = df1[df1['movieId'].values  == i]['tag']
  dff1=dff1.tolist()
  #dff1 = list(dict.fromkeys(dff1))
  print(i)
 # print(len(dff1))
 # print(df1['tag'].iloc[temp])
  df1.drop(df1.index[temp:temp+len(dff1)],inplace=True)
  df1['tag'].iloc[temp] = dff1
  #print(df1['tag'].iloc[temp])
  temp+=1

 

  

 

1
2
3


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/internals/blocks.py:937: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


Streaming output truncated to the last 5000 lines.
102716
102720
102735
102739
102742
102747
102749
102751
102753
102758
102760
102765
102779
102781
102783
102792
102794
102796
102800
102802
102807
102817
102819
102821
102823
102826
102843
102848
102850
102854
102856
102860
102862
102868
102870
102874
102876
102880
102897
102899
102903
102905
102910
102912
102917
102920
102922
102924
102941
102949
102956
102963
102967
102974
102976
102978
102984
102989
102991
102993
103003
103006
103010
103017
103022
103027
103030
103032
103042
103044
103048
103050
103052
103057
103059
103064
103066
103068
103071
103075
103081
103083
103085
103091
103103
103105
103107
103112
103116
103118
103120
103130
103137
103141
103150
103164
103166
103171
103175
103177
103182
103190
103203
103208
103210
103212
103219
103221
103224
103228
103233
103235
103237
103249
103253
103255
103269
103273
103275
103277
103279
103282
103286
103288
103290
103299
103306
103308
103315
103317
103319
103322
103324
103330
103335
1033

IndexError: ignored

In [ ]:
df1.head()

In [ ]:
df.join(df1,on='movieId')